In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import config
import json as json
from config import gmaps_key

Store Part I results into DataFrame
Load the csv exported in Part I to a DataFrame

In [2]:
#os.getcwd()
cities_vac_df = pd.read_csv('../output_data/cities.csv')
cities_vac_df.dropna(inplace = True) 
cities_vac_df

,Unnamed: 0,City,Lat,Lon,Temp_C,Feels_like,Temp_min_C,Temp_max_C,Humidity,Wind_speed_mph,Clouds,Country,Time_zone
0,1,marsabit,2.33,37.99,21.52,18.35,21.52,21.52,55,5.45,55,KE,10800
1,2,ribeira grande,38.52,-28.70,23.00,23.10,23.00,23.00,64,2.60,40,PT,0
2,3,tuktoyaktuk,69.45,-133.04,13.00,6.82,13.00,13.00,58,7.20,40,CA,-21600
3,4,ocos,14.51,-92.19,31.53,35.41,31.00,32.00,66,3.10,40,GT,-21600
4,5,cayenne,4.93,-52.33,31.00,32.35,31.00,31.00,58,4.60,52,GF,-10800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,571,lyngseidet,69.58,20.22,9.00,7.73,9.00,9.00,100,1.50,75,NO,7200
531,572,goundam,16.41,-3.67,39.53,37.34,39.53,39.53,21,4.50,0,ML,0
532,573,muscle shoals,34.74,-87.67,32.90,36.40,32.22,33.89,52,1.50,1,US,-18000
533,574,oksovskiy,62.61,39.90,14.30,13.83,14.30,14.30,94,2.17,100,RU,10800


Humidity Heatmap
Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gmaps_key)
locations = cities_vac_df[["Lat", "Lon"]].astype(float)
humidity = cities_vac_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

Create new DataFrame fitting weather criteria
Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [4]:
#This is kind of bogus because data are for one timepoint only... Could be a sunny day in Bergen, it happens.
cities_pref_df = cities_vac_df
cities_pref_df.drop(cities_pref_df[cities_pref_df['Feels_like'] > 30].index, inplace = True)
cities_pref_df.drop(cities_pref_df[cities_pref_df['Feels_like'] < -10].index, inplace = True)
cities_pref_df

,Unnamed: 0,City,Lat,Lon,Temp_C,Feels_like,Temp_min_C,Temp_max_C,Humidity,Wind_speed_mph,Clouds,Country,Time_zone
0,1,marsabit,2.33,37.99,21.52,18.35,21.52,21.52,55,5.45,55,KE,10800
1,2,ribeira grande,38.52,-28.70,23.00,23.10,23.00,23.00,64,2.60,40,PT,0
2,3,tuktoyaktuk,69.45,-133.04,13.00,6.82,13.00,13.00,58,7.20,40,CA,-21600
5,6,mount gambier,-37.83,140.77,3.00,-1.02,3.00,3.00,86,3.10,99,AU,34200
6,7,rikitea,-23.12,-134.97,19.40,14.56,19.40,19.40,64,7.98,0,PF,-32400
...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,568,tomohon,1.33,124.84,24.00,28.53,24.00,24.00,94,1.00,20,ID,28800
528,569,csorvas,46.63,20.83,21.72,21.72,20.56,23.00,64,2.10,42,HU,7200
530,571,lyngseidet,69.58,20.22,9.00,7.73,9.00,9.00,100,1.50,75,NO,7200
533,574,oksovskiy,62.61,39.90,14.30,13.83,14.30,14.30,94,2.17,100,RU,10800


Hotel Map
Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [11]:
#target_coordinates = "43.6187102, -116.2146068"

target_radius = 5000
target_type = "hotel"

params = {
    "radius": target_radius,
    "type": target_type,
    "key": gmaps_key
}


for index, row in cities_pd.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lon"]

    params["location"] = f"{lat},{lon}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)

    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))








    
    
    
    
    #print(response.url)


# use iterrows to iterate through pandas dataframe

#for index, row in #types_df.iterrows():
#location bias: circle:radius@lat,lng
    # get restaurant type from df
#    restr_type = row['ethnicity']

    # add keyword to params dict
#    params['keyword'] = restr_type

    # assemble url and make API request
#    print(f"Retrieving Results for Index {index}: {restr_type}.")

#url = base_url + params
#url
#name_address = requests.get(base_url, params=params)
#name_address = name_address.json()
#print(json.dumps(name_address, indent=4, sort_keys=True))

 #   try:
#        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
#        types_df.loc[index, 'name'] = results[0]['name']
#        types_df.loc[index, 'address'] = results[0]['vicinity']
#       types_df.loc[index, 'price_level'] = results[0]['price_level']
#        types_df.loc[index, 'rating'] = results[0]['rating']
        
 #   except (KeyError, IndexError):
 #       print("Missing field/result... skipping.")
        
 #   print("------------")






# find the closest hotel to coordinates

#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#params = {
#    "location": "39.952583,-75.16522",  # philadelphia coords,
#    "rankby": "distance",
#    "type": "restaurant",
#    "key": gkey,
#}
# use iterrows to iterate through pandas dataframe
#for index, row in types_df.iterrows():

    # get restaurant type from df
#    restr_type = row['ethnicity']

    # add keyword to params dict
#    params['keyword'] = restr_type

    # assemble url and make API request
#    print(f"Retrieving Results for Index {index}: {restr_type}.")
#    response = requests.get(base_url, params=params).json()
    
    # extract results
#    results = response['results']
    
#    try:
#        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
#        types_df.loc[index, 'name'] = results[0]['name']
#        types_df.loc[index, 'address'] = results[0]['vicinity']
#       types_df.loc[index, 'price_level'] = results[0]['price_level']
#        types_df.loc[index, 'rating'] = results[0]['rating']
        
 #   except (KeyError, IndexError):
 #       print("Missing field/result... skipping.")
        
 #   print("------------")

{
    "html_attributions": [],
    "next_page_token": "CqQCGgEAAPDtlDkCDwkWKLr2GlDQsMvEADU4RxEyVyEDe1S6UtgZJBqyPCAXE523xeg5ndH7uVR_z5IDQT1LP7Z2kFd-sdNIkkT6srIYh6Xp-OQEuFeQWyF1cJwHl4TacOMNY66_JxgWweQh8mJRQW3mv39EHoXOQs_i07qi2q7Mlr1xeDPhF9_nqudt5QLV559v2wFPdCTJSlAwoZg9ZUUr4E2BliJoYH2mrvU7iDyYNwd3tCAUXlnQ2fzgOk0BpOaFK5S-pItyFmMsGbsD1vzn0thCDcRY31E2W0zV_6s0YKQ2CjF-K8mX37ScRbSp5BxPwlMc-N2KKzLBuDOitpnldyBb2FH91EOSRpvZ3-vlaWR0QusthdZ0qHU30ZGkTvVJSlR9whIQ9bKw5unOuzjPKCVuM1m1zhoULIgCJWQ2ExBYEyZWj5s07Hi7bLE",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 43.6150186,
                    "lng": -116.2023137
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.6898951,
                        "lng": -116.1019091
                    },
                    "southwest": {
                        "lat": 43.511717,
                        "lng": -116.3658869
                  

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure